In [1]:
import numpy as np
import h5py
import qutip as qt

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz/00004_nk_spin_chain_ghz_N3.h5','r') as hf:
#     print('List of arrays in this file: \n', hf.keys())
    

    gate_time = np.array(hf.get('total_time'))
    gate_steps = np.array(hf.get('steps'))
    H0 = np.array(hf.get('H0'))
    Hops = np.array(hf.get('Hops'))
    initial_vectors_c = np.array(hf.get('initial_vectors_c'))
    uks = np.array(hf.get('uks'))[-1]
    
    inter_vecs_raw_real = np.array(hf.get('inter_vecs_raw_real'))[-1]
    inter_vecs_raw_imag = np.array(hf.get('inter_vecs_raw_imag'))[-1]
    
    inter_vecs_raw = inter_vecs_raw_real + 1j*inter_vecs_raw_imag
    
    
#     print inter_vecs_raw[0]
#     print inter_vecs_raw[1][1]
   

('List of arrays in this file: \n', [u'H0', u'Hnames', u'Hops', u'U', u'convergence', u'dressed_info', u'error', u'initial_vectors_c', u'inter_vecs_imag', u'inter_vecs_mag_squared', u'inter_vecs_raw_imag', u'inter_vecs_raw_real', u'inter_vecs_real', u'iteration', u'maxA', u'method', u'reg_coeffs', u'reg_error', u'run_time', u'states_concerned_list', u'steps', u'taylor_scaling', u'taylor_terms', u'total_time', u'uks', u'unitary_scale'])


In [16]:
print len(initial_vectors_c)

1


In [3]:
init_vector_id = 0

In [4]:
psi0 = qt.Qobj(initial_vectors_c[init_vector_id])
H0_qobj = qt.Qobj(H0)
Hops_qobj = []

for Hop in Hops:
    Hops_qobj.append(qt.Qobj(Hop))

In [5]:
tlist = np.linspace(0,gate_time,gate_steps+1)

In [8]:
uks_t0 = np.zeros((uks.shape[0],1))

uks = np.hstack([uks,uks_t0])

In [9]:
dt = gate_time/gate_steps

In [10]:
def make_get_uks_func(id):
    def _function(t,args=None):
        time_id = int(t/dt)
        return uks[id][time_id]
    return _function

In [11]:
Ht_list = []
Ht_list.append(H0_qobj)

for ii in range(len(Hops)):
    Ht_list.append([Hops_qobj[ii],make_get_uks_func(ii)])

In [12]:
output = qt.sesolve(Ht_list, psi0, tlist, [])

In [13]:
state_tlist = []
for state in output.states:
    state_tlist.append(state.full())
    
state_tlist = np.array(state_tlist)[:,:,0]
state_tlist = np.transpose(state_tlist)

In [14]:
diff = np.abs(state_tlist) - np.abs(inter_vecs_raw[init_vector_id])
print np.max(diff)

0.000187893986802


In [15]:
np.allclose(state_tlist,inter_vecs_raw[init_vector_id],atol=1e-03)

True